# Assignment #4-5: Anonymising Textual Data and De-Anonymisation
- Dataset:  Tweets Emotions [Daset](https://www.kaggle.com/datasets/pashupatigupta/emotion-detection-from-text?resource=download)
- Credits: Dataset was put together by Pashipatu Gupta
- ToDo: To run the jupyter notebook the requirements.txt need be installed (`pip install -r requirements.txt`)

### 3.1.1 Do some research to determine what needs to be anonymised in the data and why.
- For a better understanding of the structure of the dataset , we display the attribute values
    - What columns does the dataset contain and in what format are the attribute values?
        - Therefore, each column and the first value of each column (which is not empty or Null) is printed

In [3]:
import pandas as pd

dataframe = pd.read_csv("tweet_emotions.csv")

print(dataframe.iloc[:10])

     tweet_id   sentiment                                            content
0  1956967341       empty  @tiffanylue i know  i was listenin to bad habi...
1  1956967666     sadness  Layin n bed with a headache  ughhhh...waitin o...
2  1956967696     sadness                Funeral ceremony...gloomy friday...
3  1956967789  enthusiasm               wants to hang out with friends SOON!
4  1956968416     neutral  @dannycastillo We want to trade with someone w...
5  1956968477       worry  Re-pinging @ghostridah14: why didn't you go to...
6  1956968487     sadness  I should be sleep, but im not! thinking about ...
7  1956968636       worry               Hmmm. http://www.djhero.com/ is down
8  1956969035     sadness            @charviray Charlene my love. I miss you
9  1956969172     sadness         @kelcouch I'm sorry  at least it's Friday?


### 3.1.2 Using a Natural Language Processing library (e.g. Python’s spaCy), analyse the text to identify elements of personally identifiable information (PII).



### 3.1.3 Using the techniques you applied in Assignment #1, apply a masking or transformation mechanism to modify the detected PII elements and substitute with suitable replacements.

### 3.1.4 Analyse the text to determine what if any information can be obtained after the transformation process. What conclusions can you draw from this?